In [1]:
import pickle
import pandas as pd
import optuna
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight

In [2]:
path="../"

In [3]:
train = pd.concat(objs=[pd.read_csv(path+'ddi2013-type/train.tsv', sep='\t'),
                        pd.read_csv(path+'ddi2013-type/DDI_embedding.csv'),
                        pd.read_csv(path+'ddi2013-type/DDI_sinonimi.csv')],
                  ignore_index=True).sample(frac=1)
dev = pd.read_csv(path+'ddi2013-type/dev.tsv', sep='\t')
test_org = pd.read_csv(path+'ddi2013-type/test.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"ddi2013-type/DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"ddi2013-type/DDI_embedding_test.csv")

In [4]:
test = pd.concat(objs=[pd.read_csv(path+'ddi2013-type/DDI_embedding_test.csv'),
                       test_org,
                       pd.read_csv(path+'ddi2013-type/DDI_sinonimi_test.csv')],
                 ignore_index=True)

In [5]:
with open(path+"word2index.pkl", 'rb') as output:
    w2i = pickle.load(output)
with open(path+"embedding_matrix.pkl", 'rb') as output:
    embedding_matrix = pickle.load(output)

In [6]:
categories = [['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int']]

my_text_to_word_sequence = lambda sen: keras.preprocessing.text.text_to_word_sequence(sen,
                                                                                      filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                                                      lower=True)

In [7]:
five_hot_train = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  train.label.to_numpy().reshape(-1, 1))

sentences_train = [my_text_to_word_sequence(sentence) for sentence in train['sentence']]

In [8]:
five_hot_dev = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  dev.label.to_numpy().reshape(-1, 1))

sentences_dev = [my_text_to_word_sequence(sentence) for sentence in dev['sentence']]

In [9]:
five_hot_test_org = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test_org.label.to_numpy().reshape(-1, 1))

sentences_test_org = [my_text_to_word_sequence(sentence) for sentence in test_org['sentence']]

In [10]:
five_hot_test = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test.label.to_numpy().reshape(-1, 1))

sentences_test = [my_text_to_word_sequence(sentence) for sentence in test['sentence']]

In [11]:
five_hot_sin = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_sinonimi.label.to_numpy().reshape(-1, 1))

sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

In [12]:
five_hot_emb = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_embedding.label.to_numpy().reshape(-1, 1))

sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

In [13]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences_train):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_dev):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
  
print(f'Il massimo è {max}')

Il massimo è 92


In [14]:
embedded_trainset = np.zeros(shape=(len(sentences_train), max, 300))
for i, sentence in enumerate(sentences_train):
    for j, word in enumerate(sentence):
        try:
            embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [15]:
embedded_devset = np.zeros(shape=(len(sentences_dev), max, 300))
for i, sentence in enumerate(sentences_dev):
    for j, word in enumerate(sentence):
        try:
            embedded_devset[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [16]:
embedded_testset_org = np.zeros(shape=(len(sentences_test_org), max, 300))
for i, sentence in enumerate(sentences_test_org):
    for j, word in enumerate(sentence):
        try:
            embedded_testset_org[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [17]:
embedded_testset = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
    for j, word in enumerate(sentence):
        try:
            embedded_testset[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [18]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
    for j, word in enumerate(sentence):
        try:
            embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

In [19]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
    for j, word in enumerate(sentence):
        try:
            embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
        except KeyError:
            pass

# Model

In [20]:
best_params = optuna.load_study(study_name="DDI",
                                storage="sqlite:///"+path+"ddi2013-type/optuna_ddi_studio_0.db").best_params

In [22]:
print(f'{best_params}')

{'batch_size': 89, 'dropout': 0.63, 'units': 81}


In [21]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params['units'],
                                                             recurrent_dropout=best_params['dropout'],
                                                             activation='tanh')))

model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [23]:
result = model.fit(embedded_trainset,
                   five_hot_train,
                   validation_data=(embedded_devset, five_hot_dev),
                   epochs=100,
                   batch_size=best_params['batch_size'],
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                            patience=10,
                                                            restore_best_weights=True)])

Epoch 1/100
633/633 [==============================] - 62s 98ms/step - loss: 0.2623 - accuracy: 0.8919 - val_loss: 0.4926 - val_accuracy: 0.7420
Epoch 2/100
633/633 [==============================] - 72s 114ms/step - loss: 0.2530 - accuracy: 0.8949 - val_loss: 0.5883 - val_accuracy: 0.7392
Epoch 3/100
633/633 [==============================] - 125s 198ms/step - loss: 0.2473 - accuracy: 0.8983 - val_loss: 0.5361 - val_accuracy: 0.7372
Epoch 4/100
633/633 [==============================] - 127s 200ms/step - loss: 0.2385 - accuracy: 0.9026 - val_loss: 0.5976 - val_accuracy: 0.7374
Epoch 5/100
633/633 [==============================] - 127s 201ms/step - loss: 0.2329 - accuracy: 0.9054 - val_loss: 0.6124 - val_accuracy: 0.7435
Epoch 6/100
633/633 [==============================] - 127s 201ms/step - loss: 0.2252 - accuracy: 0.9077 - val_loss: 0.7035 - val_accuracy: 0.7394
Epoch 7/100
633/633 [==============================] - 127s 200ms/step - loss: 0.2199 - accuracy: 0.9099 - val_loss: 0.56

In [24]:
model.save_weights('DDI3_005.h5')

# EVALUATION

In [25]:
def print_confusionMatrix_fscore(prediction, hot_encoding):
    cm = confusion_matrix(hot_encoding.argmax(axis=1), prediction.argmax(axis=1))
    fscore = f1_score(y_true=hot_encoding.argmax(axis=1),
                      y_pred=prediction.argmax(axis=1),
                      average=None)

    print('Confusion Matrix:\n\t\tDDI-false\tDDI-mechanism\tDDI-effect\tDDI-advise\tDDI-int')
    print(f'DDI-false\t{cm[0][0]}\t\t{cm[0][1]}\t\t{cm[0][2]}\t\t{cm[0][3]}\t\t{cm[0][4]}')
    print(f'DDI-mechanism\t{cm[1][0]}\t\t{cm[1][1]}\t\t{cm[1][2]}\t\t{cm[1][3]}\t\t{cm[1][4]}')
    print(f'DDI-effect\t{cm[2][0]}\t\t{cm[2][1]}\t\t{cm[2][2]}\t\t{cm[2][3]}\t\t{cm[2][4]}')
    print(f'DDI-advise\t{cm[3][0]}\t\t{cm[3][1]}\t\t{cm[3][2]}\t\t{cm[3][3]}\t\t{cm[3][4]}')
    print(f'DDI-int\t\t{cm[4][0]}\t\t{cm[4][1]}\t\t{cm[4][2]}\t\t{cm[4][3]}\t\t{cm[4][4]}')

    zipped_fscore = zip(['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int'], fscore)
    print('\n\nFSCORE:')
    for nm, val in zipped_fscore:
        print(f'{nm}: {val}')

In [26]:
from sklearn.metrics import confusion_matrix, f1_score

## DATASET ORIGINARIO

In [27]:
result_base=model.evaluate(embedded_trainset, five_hot_train, batch_size=best_params['batch_size'])
print(f'DATASET ORIGINARIO{result_base}')

633/633 [==============================] - 27s 42ms/step - loss: 0.1707 - accuracy: 0.9306
DATASET ORIGINARIO[0.17071473598480225, 0.9305784702301025]


In [28]:
pred = model.predict(embedded_trainset, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_train)

Confusion Matrix:
		DDI-false	DDI-mechanism	DDI-effect	DDI-advise	DDI-int
DDI-false	46812		268		258		140		48
DDI-mechanism	1011		1803		24		0		0
DDI-effect	1312		14		2294		10		6
DDI-advise	681		0		2		1216		0
DDI-int		137		0		0		0		301


FSCORE:
DDI-false: 0.9604530206506017
DDI-mechanism: 0.7324801950030468
DDI-effect: 0.7383327969102027
DDI-advise: 0.7448698315467075
DDI-int: 0.759142496847415


## TESTSET

In [29]:
result_base=model.evaluate(embedded_testset, five_hot_test, batch_size=best_params['batch_size'])
print(f'DATASET TEST{result_base}')

195/195 [==============================] - 8s 41ms/step - loss: 0.6194 - accuracy: 0.8207
DATASET TEST[0.6193644404411316, 0.8207486867904663]


In [ ]:
pred = model.predict(embedded_testset, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_test)

## TESTSET ORIGINARIO

In [30]:
result_base=model.evaluate(embedded_testset_org, five_hot_test_org, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

65/65 [==============================] - 2s 28ms/step - loss: 0.5901 - accuracy: 0.8164
DATASET TEST[0.5900827646255493, 0.8163513541221619]


In [31]:
pred = model.predict(embedded_testset_org, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_test_org)

Confusion Matrix:
		DDI-false	DDI-mechanism	DDI-effect	DDI-advise	DDI-int
DDI-false	4343		151		203		78		7
DDI-mechanism	187		111		4		0		0
DDI-effect	183		2		171		4		0
DDI-advise	142		0		2		77		0
DDI-int		78		1		16		0		1


FSCORE:
DDI-false: 0.8940813175501802
DDI-mechanism: 0.3915343915343915
DDI-effect: 0.4523809523809524
DDI-advise: 0.4052631578947368
DDI-int: 0.019230769230769232


## DATASET SINONIMI

In [32]:
result_base=model.evaluate(embedded_sin, five_hot_sin, batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

65/65 [==============================] - 2s 28ms/step - loss: 0.6793 - accuracy: 0.8200
DATASET SINONIMI[0.6792577505111694, 0.8199965357780457]


In [33]:
pred = model.predict(embedded_sin, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_sin)

Confusion Matrix:
		DDI-false	DDI-mechanism	DDI-effect	DDI-advise	DDI-int
DDI-false	4508		92		107		65		10
DDI-mechanism	239		48		13		1		1
DDI-effect	259		3		91		6		1
DDI-advise	139		1		5		76		0
DDI-int		86		2		7		0		1


FSCORE:
DDI-false: 0.9004294417257566
DDI-mechanism: 0.21428571428571427
DDI-effect: 0.31217838765008576
DDI-advise: 0.41192411924119243
DDI-int: 0.018348623853211007


## DATASET EMBEDDING

In [34]:
result_base=model.evaluate(embedded_emb, five_hot_emb, batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

65/65 [==============================] - 2s 28ms/step - loss: 0.6436 - accuracy: 0.7971
DATASET SINONIMI[0.643576979637146, 0.797083854675293]


In [35]:
pred = model.predict(embedded_emb, batch_size=best_params['batch_size'])

print_confusionMatrix_fscore(prediction=pred, hot_encoding=five_hot_emb)

Confusion Matrix:
		DDI-false	DDI-mechanism	DDI-effect	DDI-advise	DDI-int
DDI-false	4281		183		194		118		6
DDI-mechanism	209		90		2		1		0
DDI-effect	203		5		146		5		1
DDI-advise	140		0		6		75		0
DDI-int		78		0		18		0		0


FSCORE:
DDI-false: 0.8833178582482204
DDI-mechanism: 0.3103448275862069
DDI-effect: 0.40220385674931125
DDI-advise: 0.35714285714285715
DDI-int: 0.0
